In [ ]:
import mlflow
import os
from mlflow.client import MlflowClient

# --- CONFIG ---
USERNAME = "Akashthakur0326"
TOKEN = "151e112b65246898b07ded104b6490eb8fff4fbd"
REPO_NAME = "FIND-TUNES"

os.environ["MLFLOW_TRACKING_URI"] = f"https://dagshub.com/{USERNAME}/{REPO_NAME}.mlflow"
os.environ["MLFLOW_TRACKING_USERNAME"] = USERNAME
os.environ["MLFLOW_TRACKING_PASSWORD"] = TOKEN

mlflow.set_tracking_uri(os.environ["MLFLOW_TRACKING_URI"])
client = MlflowClient()

class ModelWrapper(mlflow.pyfunc.PythonModel):
    def predict(self, context, model_input):
        return None

def register_proper_model(model_name, pth_path, onnx_path):
    print(f"📡 Registering {model_name}...")

    with mlflow.start_run() as run:
        artifacts = {
            "torch_model": pth_path,
            "onnx_model": onnx_path
        }

        # ONE-STEP LOG AND REGISTER
        # Passing registered_model_name here handles the registration automatically
        mlflow.pyfunc.log_model(
            artifact_path="model_package", # This is just a folder name in the run
            python_model=ModelWrapper(),
            artifacts=artifacts,
            registered_model_name=model_name # <--- THIS is the registration
        )

        # Wait a second for DagsHub to process the registration
        import time
        time.sleep(2)

        # Get the version and set it as champion
        versions = client.get_latest_versions(model_name, stages=["None"])
        if versions:
            latest_version = versions[0].version
            client.set_registered_model_alias(model_name, "champion", latest_version)
            print(f"👑 {model_name} v{latest_version} is now @champion\n")
        else:
            print(f"⚠️ Registration confirmed, but version not found yet for {model_name}.")

# --- EXECUTE ---
register_proper_model(
    "CRNN_Pitch",
    "/content/drive/MyDrive/FIND_TUNE/pitch_based_model/finetuned_models(BEST)/FINETUNED_CRNN_Smooth.pth",
    "/content/drive/MyDrive/FIND_TUNE/onnx_models/pitch_crnn.onnx"
)

register_proper_model(
    "Siamese_Spectrogram",
    "/content/drive/MyDrive/FIND_TUNE/spectrogram_based_model/shallow_CNN/best_model.pth",
    "/content/drive/MyDrive/FIND_TUNE/onnx_models/spectrogram_cnn.onnx"
)